# Site selection protocol for the i-trees project

As part of the i-Trees project a major data collection campaign will be started in 2023. Over the course of the summer the cooling effect of different tree species will be assessed in 6 different cities in the Netherlands.

There are tens of thousands of trees under municipal management in the respective cities. This poses the question of how to select the trees to be studied. Additional requirements for the selection are that trees in each city should be similarly situated. This prompts the need for a reproduceable approach for tree selection (hereafter the tree selection protocol).

The tree selection follows different tracks and is applied in a step by step process:

1. All trees should be at least 25m away from water surfaces

then, further 

2. trees in parks.
3. trees in street canyons.

for both avenues, different criteria area applied. 

__Trees in parks:__

- trees shall be at least x m away from other landuse
- trees shall be isolated from other trees (free standing), which is assessed by calculating the crown cover % in a x m buffer around each tree.
- trees shall be in a specific density environment which is assessed by calculating the Floor Space Index (a built up area density measure) in a x m buffer around it.

__Trees in street canyons:__

- trees shall be located in streets that are oriented e-w (within a healthy margin) so building shadows don't interfere with the measurements.
- trees shall be located in streets with a width >= x.
- trees shall be located in streets with a h/w ratio of <= x.
- trees shall be isolated from other trees (free standing), which is assessed by calculating the crown cover % in a x m buffer around each tree.

__FSI/GSI in buffer around trees__
- FSI/GSI were calculated, threshold values still need to be defined

---------

If not already done, please install all libraries and dependencies using the environment.txt file using:

$ conda create -n <environment-name> --file req.txt

<div>
<img src="../documents/protocol.jpeg" width="1000"/>
</div>

## Section 0: Import modules and functions

### Section 0.1: Modules

In [1]:
import pandas as pd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt

import numpy as np
from tqdm.auto import tqdm  # progress bar
import math
import warnings

import geopandas as gpd
import libpysal
import networkx as nx
import numpy as np
from numpy.lib import NumpyVersion
from shapely.geometry import Point, LineString
from shapely.geometry import Polygon

import folium

#import os; os.environ['PROJ_LIB'] = '/path/to/env/share/proj'  #- not necessary if in own environment, use this if you are using gds8 and are getting an error. This points to the correct projection library.

pd.set_option('display.max_columns', None) #-- to show all columns of datasets. To reset: pd.reset_option(“max_columns”)

/opt/anaconda3/envs/momepy_env_2/lib/python3.11/site-packages/momepy/coins.py:15: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# Reimport geopandas to load it properly.
import geopandas as gpd

### Section 0.2: Functions

In [33]:
def water_exclusion(water_gdf,trees_gdf,buffersize):
    '''Filters out points within a given proximity to water surfaces and returns a filtered dataframe.
    
    Uses a dataframe with water surfaces (e.g., from landuse data),
    and a dataframe containing point data representing trees
    as well as a buffersize as input.
    '''
    # Create a copy of the geodataframe to work with
    water_buffer = water_gdf.copy()
    # Buffer the existing geometry column.
    water_buffer['geometry'] = water_buffer.buffer(buffersize)
    # water_gdf_dissolved = water_gdf.dissolve()
    trees_no_water = trees_gdf.overlay(water_buffer, how='difference')
    print(f'The water filter removed {len(trees_gdf)-len(trees_no_water)} trees from the dataset which were less than {buffersize}m away from a water surface. There are {len(trees_no_water)} trees left.')
    return trees_no_water

def tree_exclusion(parks_gdf,tree_gdf,buffersize,tolerance):
    '''TO-DO: Write documentation for function'''
    # Create a copy of the geodataframe to work with
    tree_buffer = tree_gdf.copy()
    # Buffer the existing geometry column.
    tree_buffer['geometry'] = tree_buffer.buffer(buffersize)

    # Grab a single tree using the sample function
    single_tree = tree_buffer.sample()
    # Use that tree to store the area of the buffer polygon to filter the results later.
    circle_area = float((single_tree.geometry.area * tolerance))

    # Use the overlay function to calculate the intersection between tree_buffer and park polygons. 
    park_intersect = tree_buffer.overlay(parks_gdf, how='intersection')
    # Calculate the area (in m2) that is left of each individual tree buffer after the intersection
    park_intersect['buffer_area'] = park_intersect.geometry.area
    
    # See which of the trees is at least the buffersize away from the park boundaries (and thus other land use)
    tree_selection = park_intersect.loc[park_intersect.buffer_area >= circle_area]
    print(f'The tree filter removed {len(tree_gdf)-len(tree_selection)} trees from the dataset which were less than {buffersize}m away from a the park boundaries. There are {len(tree_selection)} trees left.')
    return tree_selection

def calculatetreecrownareainvariablebuffer(selection_gdf, tree_gdf, buffersize):
    """
    Using the tree_selection dataset from before (trees away from other landuse), the dataset of all trees in parks, and a buffer size in meters as input, calculates the tree crown area in the buffer.
    
    Creates a buffer, then conducts spatial intersection using spatial index for candidates GeoDataFrame to make queries faster.
    
    Clips candidate geometries with buffer area, then computes additional variables in clip area
    
    Finally merges newly computed variables to the selection_gdf for return (adds columns to selection_df).
    """
    # Creating the basic variables
    source_sindex = selection_gdf.sindex
    possible_matches_index = []
    result = {}
    
    # Create a copy
    
    intersecting_gdf = selection_gdf.copy()

    #Creating the buffers
    intersecting_gdf['geometry'] = intersecting_gdf.buffer(buffersize)
    
    
    # Create buffer from radius value in tree dataset
    # tree_gdf['buffer'] = tree_gdf.buffer((tree_gdf['bm_crown_diameter']/2))
    # tree_buffers = tree_gdf.loc[:,('uuid','buffer')]
    # tree_buffers.rename(columns={'buffer':'geometry'}, inplace=True)
    # tree_sindex = tree_buffers.sindex
    
    tree_buffers = tree_gdf.copy()
    
    tree_buffers['geometry'] = tree_buffers.buffer((tree_gdf['bm_crown_diameter']/2))
    
    tree_sindex = tree_buffers.sindex
        
    
        # 'itertuples()' function is a faster version of 'iterrows()'
    for other in tqdm(intersecting_gdf.itertuples(), total=intersecting_gdf.shape[0]):
        treeid = other.rowid
        bounds = other.geometry.bounds
        boundaries = other.geometry
        tree_area_list = []

        # Get unique candidates
        unique_candidate_matches = list(tree_sindex.intersection(bounds))
        possible_matches = tree_buffers.iloc[unique_candidate_matches]

        # Conduct the actual intersect
        matches = possible_matches.loc[possible_matches.intersects(other.geometry)]
        
        
        #matches = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]
        
        # Using the results of the intersect, clip the possible matches with the current iteration of boundaries from intersecting gdf. 
            
        clip = matches.clip(boundaries)
        
        tree_area_list = clip.geometry.area
        
        tree_crown_area = sum(tree_area_list)
        
        # print(f'The area of the tree crowns is {tree_crown_area}')
        
        result[treeid] = tree_crown_area
        
    tree_crown_area_df = pd.DataFrame.from_dict(result, orient = 'index', columns=[f'tree_crown_area_{buffersize}m'])
    
    merged_df = pd.merge(selection_gdf, tree_crown_area_df, left_on='rowid', right_index=True)
    
    #merged_df = merged_df.drop(columns=['buffer'])
    
    # Grab a single tree using the sample function
    single_tree = merged_df.sample()
    
    single_tree_buffer = single_tree.buffer(buffersize)
    # Use that tree to store the area of the buffer polygon to filter the results later.
    circle_area = float(single_tree_buffer.geometry.area)
    
    merged_df[f'perc_tree_cover_{buffersize}m'] = merged_df[f'tree_crown_area_{buffersize}m']/circle_area
    
    return merged_df
                                                                                   

# Taken straight from momepy, needed for orientation calculation
def _azimuth(point1, point2):
    """azimuth between 2 shapely points (interval 0 - 180)
    
    Taken straight from momepy, to use in orientation function below:
    
    https://github.com/pysal/momepy/blob/f16598cddf2a82f1a865c7bebbe61ed53d074862/momepy/utils.py#L477
    """
    angle = np.arctan2(point2[0] - point1[0], point2[1] - point1[1])
    return np.degrees(angle) if angle > 0 else np.degrees(angle) + 180

# Adapted from momepy to not show deviation from cardinal directions, but deviation from 0. .
def orientation(gdf, verbose=True):
    '''Calculate the orientation of object
    
    Captures the deviation of orientation in degrees (0-180).
    Orientation of LineStrings is represented by the orientation of line
    connecting first and last point of the segment.
    
    Adapted from momepy to not show deviation from cardinal directions 
    Source: https://github.com/pysal/momepy/blob/v0.5.4/momepy/distribution.py#L32-L96
    
    Also replaced geom.type with geom.geom_type to avoid futurewarning.
    '''
    #self.gdf = gdf
    # define empty list for results
    results_list = []

    def _dist(a, b):
        return math.hypot(b[0] - a[0], b[1] - a[1])

    for geom in tqdm(gdf.geometry, total=gdf.shape[0], disable=not verbose):
        if geom.geom_type in ["Polygon", "MultiPolygon", "LinearRing"]:
            # TODO: vectorize once minimum_rotated_rectangle is in geopandas
            bbox = list(geom.minimum_rotated_rectangle.exterior.coords)
            axis1 = _dist(bbox[0], bbox[3])
            axis2 = _dist(bbox[0], bbox[1])

            if axis1 <= axis2:
                az = _azimuth(bbox[0], bbox[1])
            else:
                az = _azimuth(bbox[0], bbox[3])
        elif geom.geom_type in ["LineString", "MultiLineString"]:
            coords = geom.coords
            az = _azimuth(coords[0], coords[-1])
        else:
            results_list.append(np.nan)
            continue

        results_list.append(az)

    # get a deviation from cardinal directions
    #results = np.abs((np.array(results_list, dtype=float)))# + 45) % 90 - 45)
    #results = np.abs(np.array(results_list, dtype=float))
    results = np.array(results_list, dtype=float)
    
    return pd.Series(results, index=gdf.index)
    #self.series = pd.Series(results, index=gdf.index)
    
def street_profile_nl(buildings_gdf, streets_gdf, study_areas_gdf):
    '''
    Calculates the street profile (width, openness, heights, w/h ratio) of roads in a dutch municipality. 
    Needs a dataframe containing roads, 
    a dataframe with building polygons (including height as a column), 
    and a dataframe containing the study area as input. 
    Saves the results in a dictionary of dataframes with the municipality name as key.
    Care: the dictionary street_profiles needs to be created before calling the function. 
    '''
    # Loop through the study_areas_gdf to get results for each of the cities individually
    
    for index, row in tqdm(study_areas_gdf.iterrows(), total=study_areas_gdf.shape[0]):
        
        # Grab the area name to use later
        area_name = row['NAME_2']
        
        # 's-Gravenhage has a special character in the name which needs to be removed
        if area_name == "\'s-Gravenhage":
            area_name = area_name[1:]
            
        print(f'Working on {area_name}')
        print('----------------------------')
        
        # Clip the building dataset to only include the city we're working on
        buildings = buildings_gdf.clip(row.geometry)
        print(f'Step 1: Clipped buildings of {area_name}')
        
        # Clip the roads dataset to only include the city we're working on
        roads = streets_gdf.clip(row.geometry)
        print(f'Step 2: Clipped roads of {area_name}')

        # Create a copy to avoid the settings with copy warning
        edges = roads.copy()
        
        # To calculate street orientation and street profile the network has to be exploded to get rid of multi-geometries.
        edges = edges.explode(index_parts=True)
        
        # Call the street profule function from momepy. The building footprints have got height information in the 'height_70p' column.
        profile = momepy.StreetProfile(edges, buildings, heights='height_70p')
        print(f'Step 3: Calculated street profile of {area_name}')

        # Transfer the results of the StreetProfile function to the street dataframe.
        edges['widths'] = profile.w
        edges['width_deviations'] = profile.wd
        edges['openness'] = profile.o
        edges['heights'] = profile.h
        edges['heights_deviations'] = profile.hd
        edges['profile'] = profile.p
        
        # Calculate the orientation of each linestring using the customized momepy function.
        edges['orientation'] = orientation(edges)
        print(f'Step 4: Calculated street orientation of {area_name}')
        
        # Clean up the dataset and keep only the relevant columns
        edges = edges[['STT_NAAM', 'GME_NAAM','widths', 'width_deviations', 'openness', 'heights', 'heights_deviations', 'profile', 'orientation', 'geometry']]
        print('Cleaning up...')
        
        # Save the StreetProfile of the current city in the street_profiles dictionary using the city name as key and dataframe as values.
        street_profiles[area_name] = edges
        print(f'Step 5: Stored street profile values in \'street_profiles[{area_name}]\'.')
        print('----------------------------')
    print('MISSION SUCCESS - END')


def floorspaceindexinbuffer(study_areas_gdf, tree_dict, buildings_gdf, buffersize):
    '''
    Calculates the Floor Space Index in a given buffer area.
    
    Takes a dataframe containing municipalities of the Netherlands (study_areas_gdf),
    A dictionary containing trees for the respective municipalities (tree_dict)
    As well as a dataframe containing building polygons in all study_areas (buildings_gdf)
    And finally a buffersize in m.
    
    Loops through all study areas,
    clips buildings
    extracts tree dataframe
    calculates number of floors
    calculates FSI for each tree.
    
    Saves the results in the trees_canyons_dict dictionary of dataframes with the municipality name as key.
    '''
    for index, row in tqdm(study_areas_gdf.iterrows(), total=study_areas_gdf.shape[0]):
        area_name = row['NAME_2']
        if area_name == "\'s-Gravenhage":
            area_name = area_name[1:]
        print(f'Working on {area_name}')
        print('----------------------------')
        buildings = buildings_gdf.clip(row.geometry)
        print(f'Step 1: Clipped buildings of {area_name}')
        trees = tree_dict[f'{area_name}']
        print(f'Step 2: Extracted trees of {area_name} from the dictionary')
    
        # Creating the basic variables
        buildings_sindex = buildings.sindex
        possible_matches_index = []
        result = {}
        print(f'Step 3: Created spatial index for {area_name} buildings')

        # Calculate the number of floors (rudimentary method - originally with 2.65, now with 3.15, confirm with Daniela)
        buildings['no_floors'] = np.floor((buildings['height_70p'] / 3.15))

        # Rounding down causes small buildings to be assigned 0 floors. This is to assure every building has got at least 1 floor.
        buildings.loc[buildings['no_floors'] < 1, 'no_floors'] = 1
        
        print('Step 4: Calculated number of floors and cleaned the dataset')

        # Create a copy of the tree dataset
        intersecting_gdf = trees.copy()

        #Creating the buffers
        intersecting_gdf['geometry'] = intersecting_gdf.buffer(buffersize)

        print('Step 5: For each tree, calculate FSI')
        for tree_buffer in tqdm(intersecting_gdf.itertuples(), total=intersecting_gdf.shape[0]):
            treeid = tree_buffer.rowid
            bounds = tree_buffer.geometry.bounds
            boundaries = tree_buffer.geometry
            floor_space_list = {}


            # Get unique candidates
            unique_candidate_matches = list(buildings_sindex.intersection(bounds))
            possible_matches = buildings.iloc[unique_candidate_matches]

            # Conduct the actual intersect
            matches = possible_matches.loc[possible_matches.intersects(tree_buffer.geometry)]


            #matches = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]

            # Using the results of the intersect, clip the possible matches with the current iteration of boundaries from intersecting gdf. 

            clip = matches.clip(boundaries)


            # Use a for loop to iterate through all buildings inside the clipped dataframe

            for index, row in clip.iterrows():

                # calculate floor space with no of floors and geometry area of each clip element
                floor_space = row.geometry.area * row['no_floors']

                # Saving the floor space index for each individual building in the clip area in the floor_space_list
                floor_space_list[row['gid']] = floor_space        

            # After iterating through all of the buildings in the clip, calculate the Floor Space Index in the buffer area. For this we take the floor space area of all buildings and divide it by the area of the buffer

            fsi_context = sum(floor_space_list.values()) / tree_buffer.geometry.area

            #Build the results dictionary with tree id and FSI in buffer
            result[treeid] = fsi_context

        #Convert dictionary into dataframe, making the id into the index and naming the column for avg_blg_height
        floor_space_buffer = pd.DataFrame.from_dict(result, orient = 'index', columns=[f'FSI_{buffersize}m'])
        print('Step 6: Add FSI column to the input dictionary')
        trees_canyons_dict[area_name] = pd.merge(trees, floor_space_buffer, left_on='rowid', right_index=True)
        print('-------')
    print('MISSION SUCCESS - END')
    
def groundspaceindexinbuffer_parks(study_areas_gdf, tree_dict, buildings_gdf, buffersize):
    '''
    Calculates the Ground Space Index in a given buffer area - for the trees in parks
    
    GSI is herein defined as building footprint/ buffer area calculated with radius x from the centroid of each tree.
    
    Takes a dataframe containing municipalities of the Netherlands (study_areas_gdf),
    A dictionary containing trees for the respective municipalities (tree_dict)
    As well as a dataframe containing building polygons in all study_areas (buildings_gdf)
    And finally a buffersize in m.
    
    Loops through all study areas,
    clips buildings
    extracts tree dataframe
    creates buffer around trees with buffersize as radius
    calculates GSI for each tree using the buffered dataframe.
    
    Saves the results in the trees_in_parks_dict dictionary of dataframes with the municipality name as key.
    '''
    for index, row in tqdm(study_areas_gdf.iterrows(), total=study_areas_gdf.shape[0]):
        area_name = row['NAME_2']
        if area_name == "\'s-Gravenhage":
            area_name = area_name[1:]
        print(f'Working on {area_name}')
        print('----------------------------')
        buildings = buildings_gdf.clip(row.geometry)
        print(f'Step 1: Clipped buildings of {area_name}')
        trees = tree_dict[f'{area_name}']
        print(f'Step 2: Extracted trees of {area_name} from the dictionary')
    
    
        # Creating the basic variables
        buildings_sindex = buildings.sindex
        possible_matches_index = []
        result = {}
        print(f'Step 3: Created spatial index for {area_name} buildings')

        # Create a copy of the tree dataset
        intersecting_gdf = trees.copy()

        #Creating the buffers
        intersecting_gdf['geometry'] = intersecting_gdf.buffer(buffersize)

        print('Step 5: For each tree, calculate GSI')
        for tree_buffer in tqdm(intersecting_gdf.itertuples(), total=intersecting_gdf.shape[0]):
            treeid = tree_buffer.rowid
            bounds = tree_buffer.geometry.bounds
            boundaries = tree_buffer.geometry
            # building_list = []


            # Get unique candidates
            unique_candidate_matches = list(buildings_sindex.intersection(bounds))
            possible_matches = buildings.iloc[unique_candidate_matches]

            # Conduct the actual intersect
            matches = possible_matches.loc[possible_matches.intersects(tree_buffer.geometry)]


            #matches = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]

            # Using the results of the intersect, clip the possible matches with the current iteration of boundaries from intersecting gdf. 

            clip = matches.clip(boundaries)
            
            clip['Area'] = clip.geometry.area
        
            #Calculate the amount of "free" space (without buildings). Taking the area of the current buffer and subtracting the are of other buildings
            #other.geometry.area is the area of the buffer which is subtracted by the geometry (footprint) area of all of the buildings that are in the buffer)
            building_footprint_area = clip['Area'].sum()

            if building_footprint_area != 0:
                gsi = building_footprint_area / tree_buffer.geometry.area
            else:
                gsi = 0        

            #Build the results dictionary with tree id and FSI in buffer
            result[treeid] = gsi

        #Convert dictionary into dataframe, making the id into the index and naming the column for avg_blg_height
        ground_space_buffer = pd.DataFrame.from_dict(result, orient = 'index', columns=[f'GSI_{buffersize}m'])
        print('Step 6: Add GSI column to the input dictionary')
        trees_in_parks_dict[area_name] = pd.merge(trees, ground_space_buffer, left_on='rowid', right_index=True)
        print('-------')
    print('MISSION SUCCESS - END')

def groundspaceindexinbuffer(study_areas_gdf, tree_dict, buildings_gdf, buffersize):
    '''
    Calculates the Ground Space Index in a given buffer area
    
    GSI is herein defined as building footprint/ buffer area calculated with radius x from the centroid of each tree.
    
    Takes a dataframe containing municipalities of the Netherlands (study_areas_gdf),
    A dictionary containing trees for the respective municipalities (tree_dict)
    As well as a dataframe containing building polygons in all study_areas (buildings_gdf)
    And finally a buffersize in m.
    
    Loops through all study areas,
    clips buildings
    extracts tree dataframe
    creates buffer around trees with buffersize as radius
    calculates GSI for each tree using the buffered dataframe.
    
     Saves the results in the trees_canyons_dict dictionary of dataframes with the municipality name as key.
    '''
    for index, row in tqdm(study_areas_gdf.iterrows(), total=study_areas_gdf.shape[0]):
        area_name = row['NAME_2']
        if area_name == "\'s-Gravenhage":
            area_name = area_name[1:]
        print(f'Working on {area_name}')
        print('----------------------------')
        buildings = buildings_gdf.clip(row.geometry)
        print(f'Step 1: Clipped buildings of {area_name}')
        trees = tree_dict[f'{area_name}']
        print(f'Step 2: Extracted trees of {area_name} from the dictionary')
    
    
        # Creating the basic variables
        buildings_sindex = buildings.sindex
        possible_matches_index = []
        result = {}
        print(f'Step 3: Created spatial index for {area_name} buildings')

        # Create a copy of the tree dataset
        intersecting_gdf = trees.copy()

        #Creating the buffers
        intersecting_gdf['geometry'] = intersecting_gdf.buffer(buffersize)

        print('Step 5: For each tree, calculate GSI')
        for tree_buffer in tqdm(intersecting_gdf.itertuples(), total=intersecting_gdf.shape[0]):
            treeid = tree_buffer.rowid
            bounds = tree_buffer.geometry.bounds
            boundaries = tree_buffer.geometry
            # building_list = []


            # Get unique candidates
            unique_candidate_matches = list(buildings_sindex.intersection(bounds))
            possible_matches = buildings.iloc[unique_candidate_matches]

            # Conduct the actual intersect
            matches = possible_matches.loc[possible_matches.intersects(tree_buffer.geometry)]


            #matches = possible_matches.loc[possible_matches.intersects(intersecting_gdf.unary_union)]

            # Using the results of the intersect, clip the possible matches with the current iteration of boundaries from intersecting gdf. 

            clip = matches.clip(boundaries)
            
            clip['Area'] = clip.geometry.area
        
            #Calculate the amount of "free" space (without buildings). Taking the area of the current buffer and subtracting the are of other buildings
            #other.geometry.area is the area of the buffer which is subtracted by the geometry (footprint) area of all of the buildings that are in the buffer)
            building_footprint_area = clip['Area'].sum()

            if building_footprint_area != 0:
                gsi = building_footprint_area / tree_buffer.geometry.area
            else:
                gsi = 0        

            #Build the results dictionary with tree id and FSI in buffer
            result[treeid] = gsi

        #Convert dictionary into dataframe, making the id into the index and naming the column for avg_blg_height
        ground_space_buffer = pd.DataFrame.from_dict(result, orient = 'index', columns=[f'GSI_{buffersize}m'])
        print('Step 6: Add GSI column to the input dictionary')
        trees_canyons_dict[area_name] = pd.merge(trees, ground_space_buffer, left_on='rowid', right_index=True)
        print('-------')
    print('MISSION SUCCESS - END')

## Section 1: Importing and cleaning tree dataset

__Don't need to do this in the final version, will receive a already filtered dataset__


In [4]:
# Read gpk
all_trees = gpd.read_file('../data/raw_data/trees/trees_11112022.gpkg')

# Checking which cities are included.
print(f'The following cities are included in the dataset {all_trees.bron.unique()}')

# Show contents
all_trees.head()

The following cities are included in the dataset ['Rotterdam' 'Utrecht' 'Den Haag' 'Groningen' 'Heerhugowaard' 'Amsterdam'
 'Hendrik-Ido-Ambacht' 'Dordrecht']


,rowid,uuid,bron,tree_name,tree_intid,genus,species,cv,bm_treeheight,bm_crown_diameter,geometry
0,1,00000a59-0b14-4fd4-8e1c-ac924d8cc9b8,Rotterdam,Aesculus hippocastanum,194,Aesculus,hippocastanum,None,25.4,11.1,POINT (87670.050 440949.360)
1,2,00000cdf-33b9-431c-84d8-be5d4cf8df87,Utrecht,Catalpa bignonioides 'Nana',349,Catalpa,bignonioides,Nana,3.0,2.0,POINT (137578.786 458618.712)
2,3,00003c8e-3f16-400d-acdc-a07705c0d22a,Den Haag,Corylus colurna 'VDB Obelisk',477,Corylus,colurna,VDB Obelisk,3.0,2.0,POINT (78722.829 452904.528)
3,4,000042b4-7bee-4999-8513-8d152afd7c43,Groningen,Salix alba,1641,Salix,alba,None,30.9,11.5,POINT (231367.398 583660.647)
4,5,00004913-6b0b-4491-8559-1a9d9443ff77,Rotterdam,Styphnolobium japonicum,1768,Styphnolobium,japonicum,None,13.5,9.8,POINT (90530.430 436378.670)


In [4]:
# Show crs of dataframe to confirm whether any transformations are necessary
all_trees.crs

<Derived Projected CRS: EPSG:28992>
Name: Amersfoort / RD New
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: Netherlands - onshore, including Waddenzee, Dutch Wadden Islands and 12-mile offshore coastal zone.
- bounds: (3.2, 50.75, 7.22, 53.7)
Coordinate Operation:
- name: RD New
- method: Oblique Stereographic
Datum: Amersfoort
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

### 1.1: Filter only necessary tree species

The tree species in question are the following:
- Platanus x hispanica
- Tilia x europaea
- Ulmus x hollandica

In [5]:
# Create a separate dataframe for each of the species in question and see how many there are 
platanus = all_trees.loc[(all_trees["genus"] == "Platanus") & (all_trees["species"] == "hispanica")]

tilia = all_trees.loc[(all_trees["genus"] == "Tilia") & (all_trees["species"] == "europaea")]

ulmus = all_trees.loc[(all_trees["genus"] == "Ulmus") & (all_trees["species"] == "hollandica")]

# Combine them into a single dataframe
species = [platanus, tilia, ulmus]
study_trees = pd.concat(species)

# Calculate the combined length of all species dataframes to get to the final figure
print(f'The total number of trees of the three selected species is: {len(study_trees)}')
print('-----------------')
print(f'This is divided into the following; Platanus: {len(platanus)}, Tilia: {len(tilia)}, Ulmus: {len(ulmus)}')

The total number of trees of the three selected species is: 133691
-----------------
This is divided into the following; Platanus: 45226, Tilia: 56250, Ulmus: 32215


#### 1.1.1 Number of trees per species per city

In [16]:
tree_observations = study_trees["genus"].value_counts()
print(tree_observations)

Tilia       56250
Platanus    45226
Ulmus       32215
Name: genus, dtype: int64


In [18]:
# Get a dataframe with the number of observations per genus per city.

obs = study_trees.groupby(["bron", "genus"]).size().reset_index(name="Times")

# They are object type, so change type to string to further work with them

obs['bron'] = obs['bron'].astype("string")
obs['genus'] = obs['genus'].astype("string")

obs

,bron,genus,Times
0,Amsterdam,Platanus,15099
1,Amsterdam,Tilia,14831
2,Amsterdam,Ulmus,13669
3,Den Haag,Platanus,4108
4,Den Haag,Tilia,10033
5,Den Haag,Ulmus,5671
6,Dordrecht,Platanus,2872
7,Dordrecht,Tilia,2174
8,Dordrecht,Ulmus,415
9,Groningen,Platanus,2625


## Section 2: Tree selection protocol

The first task is to remove all trees that are in proximity to water surfaces.

Then we need to split the tree dataset into two:
- Trees in parks
- Trees in street canyons (only E-W direction)

for that we need the national landuse dataset which is available via WFS or download for the whole country.

#### 2.0.1 Get Study Area boundaries:

We're working with large datasets (landuse) and don't need all of it. To filter the dataset we add the study area boundaries.

In [6]:
#Load study area extent - both Rdam and AMS
admin_area = gpd.read_file('../data/raw_data/admin_area/gadm41_NLD_2.json').to_crs(28992)

# Create a list containing the placenames as they are in the dataset.
# Den Haag is 's-Gravenhage' in the NL admin area dataset. Den Haag is the 'modern' abbreviation.
places_list = ['Rotterdam', 'Utrecht', '\'s-Gravenhage', 'Groningen', 'Heerhugowaard',
       'Amsterdam', 'Hendrik-Ido-Ambacht', 'Dordrecht']

study_areas = admin_area[admin_area['NAME_2'].isin(places_list)]

print(f"To confirm that this worked: We're left with the following cities {list(study_areas.NAME_2.unique())}")

# study_areas.plot()

To confirm that this worked: We're left with the following cities ['Groningen', 'Amsterdam', 'Heerhugowaard', 'Utrecht', "'s-Gravenhage", 'Dordrecht', 'Hendrik-Ido-Ambacht', 'Rotterdam']


#### 2.0.2 Downloading landuse data using WFS
In theory the data can be queried, but the standard with which it is accessed is tricky to play with. I've resorted to simply downloading the dataset from the link. In case you're interested, the WFS link and capabilities query below. 

In [ ]:
from owslib.wfs import WebFeatureService

# Put the WFS url in a variable
wfsUrl = 'https://geodata.nationaalgeoregister.nl/nwbwegen/wfs?'

# Create a WFS object
wfs = WebFeatureService(url=wfsUrl, version='2.0.0')

# Get the title from the object
print(wfs.identification.title)

# Check the contents of the WFS
print(list(wfs.contents))

#### 2.0.3 Downloading the full dataset from the NGR (nationaal georegister)

The landuse dataset is available for the entirety of the Netherlands and is also INSPIRE standardised. 

Download link:
https://www.nationaalgeoregister.nl/geonetwork/srv/eng/catalog.search#/metadata/6318009c-7b11-4ab9-9929-735584dfb692?tab=relations


In [7]:
# Import landuse dataset as geodataframe using geopandas, then transform to Ammersfoort (28992). This will take a while as it is a big file.
luseGDF = gpd.read_file('../data/raw_data/landuse/landuse.gml').to_crs(28992)

# Clip the dataset to the study areas. Using a buffer around the study area to capture adjoining landuse.
luseGDF_study_areas = luseGDF.clip(study_areas.buffer(100))
# Check how many features and attributes were loaded
print(f'The dataframe has {luseGDF_study_areas.shape[0]} records and {luseGDF_study_areas.shape[1]} columns')
print('----------------')
# Using .unique(), we can get all unique values in the description column and identify the values we're interested in
print(f'These are all the landuse categories contained in the dataset: {list(luseGDF_study_areas.description.unique())}')

# Check contents of landuse dataset. We're especially interested in the 'description' column
luseGDF_study_areas.head()

The dataframe has 9473 records and 12 columns
----------------
These are all the landuse categories contained in the dataset: ['Main road', 'Other semi built-up area', 'Wet natural area', 'Woodland', 'Other inland water', 'Rijn & Maas', 'Other agricultural usage', 'Water with recreational usage', 'Holiday recreation', 'Enclosed estuary', 'Industrial area and offices', 'Greenhouses', 'Area for daytrips', 'Sports ground (incl. car parks)', 'Car wreck site', 'Building site', 'Allotment garden', 'Residential', 'Retail trade, hotel and catering', 'Park and public garden', 'Socio-cultural facility', 'Cemetery', 'Public institutions', 'Water reservoir', 'Dumping site', 'Railroad', 'Dry natural area', 'Area for storing industrial water', 'Mining area', 'Airport', 'Ijsselmeer of Markermeer', 'Noordzee', 'Water where minerals are extracted']


,gml_id,description,name,localId,namespace,beginLifespanVersion,hilucsPresence,specificPresence,observationDate,validFrom,validTo,geometry
883,BBG2017_884,Main road,4_1_1,BBG2017_884,NL.CBS.LU.ExistingLandUseObject,None,None,None,2017-07-31,2017-04-15,2017-09-15,"MULTIPOLYGON (((112358.812 422024.122, 112347...."
50759,BBG2017_50760,Other semi built-up area,6_5,BBG2017_50760,NL.CBS.LU.ExistingLandUseObject,None,None,None,2017-07-31,2017-04-15,2017-09-15,"POLYGON ((104636.289 415714.495, 104640.019 41..."
147861,BBG2017_147862,Wet natural area,6_3_1,BBG2017_147862,NL.CBS.LU.ExistingLandUseObject,None,None,None,2017-07-31,2017-04-15,2017-09-15,"POLYGON ((105267.446 416100.918, 105267.441 41..."
125969,BBG2017_125970,Woodland,1_2,BBG2017_125970,NL.CBS.LU.ExistingLandUseObject,None,None,None,2017-07-31,2017-04-15,2017-09-15,"POLYGON ((104878.714 416440.997, 104879.254 41..."
147877,BBG2017_147878,Wet natural area,6_3_1,BBG2017_147878,NL.CBS.LU.ExistingLandUseObject,None,None,None,2017-07-31,2017-04-15,2017-09-15,"POLYGON ((105030.718 416663.815, 105030.654 41..."


### 2.1: Water proximity exclusion

To exclude water surfaces we're using the landuse dataset to identify all water surfaces.

We then define a buffer value for all water surfaces which we use to identify which trees to exclude.

After buffering, the trees that are too close to the water surfaces are eliminated using a overlay function (difference)

In [8]:
# Isolate the water surfaces from the rest of the countrywide landuse dataset, then check how many features are left. 
# The dataframe that was created will be used multiple times so keep it in mind
water = luseGDF_study_areas.loc[(luseGDF_study_areas.description == 'Other inland water') | 
                    (luseGDF_study_areas.description == 'Water with recreational usage') | 
                    (luseGDF_study_areas.description == 'Water reservoir') | 
                    (luseGDF_study_areas.description == 'Wet natural area') | 
                    (luseGDF_study_areas.description == 'Ijsselmeer of Markermeer') | 
                    (luseGDF_study_areas.description == 'Noordzee') | 
                    (luseGDF_study_areas.description == 'Waddenzee, Eems, Dollard') |
                    (luseGDF_study_areas.description == 'Rijn & Maas') |
                    (luseGDF_study_areas.description == 'Randmeer') |
                    (luseGDF_study_areas.description == 'Enclosed estuary') |
                    (luseGDF_study_areas.description == 'Water where minerals are extracted') |
                    (luseGDF_study_areas.description == 'Area for storing industrial water')
                   ]



#### Exclude water surfaces (running the function)

In [9]:
## Water proximity exclusion happens by buffering all water surfaces and overlaying with the tree dataset.
# Define a buffer area to exclude:
buffer_water = 25


# Using this function call, you can simply adjust the buffersize and have a look at what is being excluded. 
study_trees_no_water = water_exclusion(water_gdf=water, trees_gdf=study_trees,buffersize=buffer_water)

# Visualise results, this is basic because it takes too many resources otherwise. Might take a second to load. Can reactivate if necessary.
# study_trees_no_water.explore()

The water filter removed 22612 trees from the dataset which were less than 25m away from a water surface. There are 111079 trees left.


#### Display results nicely, just for Amsterdam

In [10]:
# USE THE BELOW FUNCTIONS TO DISPLAY THE BUFFER EXCLUSION IN AMSTERDAM. THE DATASET IS TOO BIG TO DISPLAY OTHERWISE. JUST MAKE SURE TO CHANGE THE BUFFER BELOW.

ams_shape = gpd.read_file('../data/raw_data/admin_area/gadm41_NLD_2.json').to_crs(28992)
ams_shape = ams_shape.loc[(ams_shape['NAME_2']  == 'Amsterdam')]

water_ams = water.clip(ams_shape)
water_ams['geometry'] = water_ams.buffer(buffer_water)

trees_ams = study_trees.clip(ams_shape)

study_trees_no_water_ams = study_trees_no_water.clip(ams_shape)

In [ ]:
m = water_ams.explore(column="description", tooltip="description", tiles="CartoDB positron", cmap="Set1"
)

trees_ams.explore(
     m=m, color='red' # use red color on all points
)

study_trees_no_water_ams.explore(
    m=m, color='green')

folium.TileLayer('CartoDB positron', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control

m  # show map

__NOTE:__

Performance of folium is limited inside of Jupyter because the prefer_canvas option only activates when changed as a global setting in a saved .html file. If you want to show this map to anyone then the best way to do this is to:

- Save the folium map using 
    "m.save('../results/output/MAP_NAME.html')"

- Then navigate to the folder and open the .html file with a text editor of your choosing.

- Once open, search for prefer_canvas and change the option from _false_ to _true_. Make sure to SAVE!

- Then open the file and view the map in your browser. 


### 2.2: Trees in parks

Trees in parks can be filtered by the same landuse dataset we used earlier.

In [11]:
# Isolate the parks and public gardens from the rest of the countrywide landuse dataset, then check how many features are left. 
# We will use the new dataframe that was created at a later stage
parks = luseGDF_study_areas.loc[luseGDF_study_areas.description == 'Park and public garden']
parks.shape

(926, 12)

#### 2.2.3 Add landuse information to all trees included in the study.

In [12]:
# Using a spatial join, we can add the corresponding landuse class (landuse.description) to each of the trees

treeslanduse = gpd.tools.sjoin(study_trees_no_water, luseGDF_study_areas, predicate="within", how="left")

# Drop extra columns to clean the dataset. We're only interested in the 'description' column.
treeslanduse = treeslanduse.drop(columns=['index_right','gml_id', 'name', 'localId', 'namespace', 'beginLifespanVersion', 'hilucsPresence', 'specificPresence', 'observationDate', 'validFrom', 'validTo'])

# Show the contents
treeslanduse.head()

# Can then as next step filter the data down to only include the trees in park and public garden. But this way we are creating a dataframe that we could use for a variety of queries and further filtering

,rowid,uuid,bron,tree_name,tree_intid,genus,species,cv,bm_treeheight,bm_crown_diameter,geometry,description
0,51,00038cf6-d99a-4d13-8702-76a6a047de62,Groningen,Platanus ×hispanica,1181,Platanus,hispanica,None,20.1,10.7,POINT (229687.409 580299.600),Residential
1,80,0005c248-f6f7-40aa-972b-4ce74bcfdad6,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,21.2,14.3,POINT (92083.890 438938.080),Main road
2,143,000a17db-c6ac-4ad5-b35e-597b61f1f896,Utrecht,Platanus ×hispanica,1181,Platanus,hispanica,None,4.7,4.7,POINT (134117.840 453505.637),Building site
3,156,000a928a-3b23-4e58-8012-b2dd6ed8c97b,Hendrik-Ido-Ambacht,Platanus ×hispanica,1181,Platanus,hispanica,None,3.0,11.2,POINT (103924.914 427853.067),Residential
4,163,000b238e-d4c7-4263-b092-ffbe2541620c,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,13.5,14.5,POINT (91426.590 437374.960),Main road


In [ ]:
# Create a new dataframe including only the trees that are located in the land use category we want
treesinparks = treeslanduse.loc[treeslanduse.description == 'Park and public garden']
# Check how many trees are located within the Park and public garden category
print(f'There area {treesinparks.shape[0]} trees in parks.')

# Visualise all trees in parks.
m = parks.explore()
treesinparks.explore(m=m, color='green')
# Show map
m

#### 2.2.4: Only select trees that are at least x m away from the polygon boundary.

This is done with the following steps:
- create a buffer of xm around each tree
- use the overlay function to calculate the intersections between tree and park polygons.
- assess the the buffer polygon areas after intersection. If they are untouched (or within the tolerance margin), the trees are located at least the buffer distance away from the park boundaries.

In [ ]:
## Select a buffer to filter the dataset with (value is in m)
park_boundary_buffer = 45

## Takes values between 0 and 1. Signifies the overlap tolerance in percent (e.g., 0.99 is a 1% overlap tolerance with other landuse, 0.9 is a 10% tolerance and so forth)
overlap_tolerance = 0.99

# Run the function
tree_selection = tree_exclusion(parks_gdf=parks,tree_gdf=treesinparks,buffersize=park_boundary_buffer,tolerance=overlap_tolerance)

## Bringing the tree selection back to a point dataframe (we created a polygon)
# Filter original tree dataframe by rowid of the tree buffer dataframe
treesinparks_selection = treesinparks[(treesinparks.rowid.isin(tree_selection.rowid))]

# Visualise the result
tree_selection.explore()

In [ ]:
m = parks.explore(column="description", tooltip="description", tiles="CartoDB positron", cmap="Set1"
)

trees_ams.explore(
     m=m, color='red' # use red color on all points
)

tree_selection.explore(
    m=m, color='green')

folium.TileLayer('CartoDB positron', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control
m.save("../results/output/ams_parks_trees.html")


m  # show map

#### 2.2.5: Crown area, % coverage

- Note: The tree in question is also counted in the % crown area - might have to be exluded?

#### Executing the tree crown area function

In [17]:
tree_buffer = 25

# Execute the function, make sure to use treesinparks_selection as the selection_gdf, because the tree_selection dataset is polygon, not point.
print('Calculating tree crown area for entire dataset')
crown_area_study_trees = calculatetreecrownareainvariablebuffer(selection_gdf=treesinparks_selection,tree_gdf=treesinparks,buffersize=tree_buffer)

# Create new dictionary for trees in parks
trees_in_parks_dict = {}

# Save trees in parks in dictionary
print('Save as separate city dataframe in dictionary')
for index, row in tqdm(study_areas.iterrows(), total=study_areas.shape[0]):
        
    # Grab the area name to use later
    area_name = row['NAME_2']
        
    # 's-Gravenhage has a special character in the name which needs to be removed
    if area_name == "\'s-Gravenhage":
        area_name = area_name[1:]
    # Clip the tree dataset to only include the city we're working on
    trees = crown_area_study_trees.clip(row.geometry)
    
    trees_in_parks_dict[area_name] = trees
    
# Visualise the crown area calculated before (activate if wanted)
#crown_area_study_trees.explore(column=f'perc_tree_cover_{tree_buffer}m',cmap='viridis',scheme='NaturalBreaks')


Calculating tree crown area for entire dataset


  0%|          | 0/2092 [00:00<?, ?it/s]

Save as separate city dataframe in dictionary


  0%|          | 0/8 [00:00<?, ?it/s]

### Section 2.3: Trees in street canyons

Process:

Identify trees in street canyons:
- For simplification we're first selecting trees that are not in parks (inverse of above function)

Then:
- we use the official street dataset of the Netherlands as well as buildings taken from the 3dbag.
- using the street network and building footprints (with height attributes), we calculate the street profile (e.g., widths, w/h ratio) and street orientation.
- we then filter by street orientation, using the orientation values and street widths. 

To-do: 
- Clean up the road datasets in the folders.


#### 2.3.1 Initial selection of trees

In [18]:
# Create a new dataframe including only the trees that are located in the land use category we want
trees_other = treeslanduse.loc[~(treeslanduse.description == 'Park and public garden')]

# Check how many trees are located within the Park and public garden category
trees_other.shape

(93157, 12)

#### 2.3.2 Street orientation and street profile (widths, h/w ratio)

Potential to-do's:
- Split further into nodes and edges by converting to graph and back (see if it makes a difference)
- Network could be simplified. This could be done using  using BGT road polygons and calculating the centreline (or just grabbing the width from the polygons?)

##### (ONLY FOR REFERENCE) Importing and clipping all roads in the NL

In [ ]:
## THIS IS ONLY FOR REFERENCE. THE ORIGINAL DATASET IS LOCATED HERE, WAS CLIPPED TO THE STUDY AREA EXTENT.
# read road network of NL
network = gpd.read_file('../data/raw_data/rijkswaterstraat/01-12-2022/Wenetworkn/Wegvakken.shp')

# Clipped to the study areas
network_study_areas = network.clip(study_areas)
# Then saved to file
network_study_areas.to_file('../data/clean_data/wegvakken.gpkg', layer='original', driver='GPKG')

##### Execute from here:

In [19]:
# Import all buildings in NL
buildings_nl = gpd.read_file('../data/clean_data/3dbag/buildings_study_area.gpkg')
# 'Clean' the building dataset, include only 'buildings' or 'structures' larger than 30 m2.
buildings_nl = buildings_nl.loc[(buildings_nl.geometry.area > 30)]

# Import the streetnetwork if you haven't already. These are large files so this will take a second.
network = gpd.read_file('../data/clean_data/wegvakken.gpkg', layer='original')

In [20]:
# Create an empty dictionary for the street profile dataframes to be saved in
street_profiles = {}

# Call the street_profile_nl function to calculate the street profiles and orientation values. Results are saved to the street_profiles dictionary
street_profile_nl(buildings_gdf=buildings_nl, streets_gdf=network,study_areas_gdf=study_areas)

  0%|          | 0/8 [00:00<?, ?it/s]

Working on Groningen
----------------------------
Step 1: Clipped buildings of Groningen
Step 2: Clipped roads of Groningen
Step 3: Calculated street profile of Groningen


  0%|          | 0/14254 [00:00<?, ?it/s]

Step 4: Calculated street orientation of Groningen
Cleaning up...
Step 5: Stored street profile values in 'street_profiles[Groningen]'.
----------------------------
Working on Amsterdam
----------------------------
Step 1: Clipped buildings of Amsterdam
Step 2: Clipped roads of Amsterdam
Step 3: Calculated street profile of Amsterdam


  0%|          | 0/35395 [00:00<?, ?it/s]

...

Step 4: Calculated street orientation of Dordrecht
Cleaning up...
Step 5: Stored street profile values in 'street_profiles[Dordrecht]'.
----------------------------
Working on Hendrik-Ido-Ambacht
----------------------------
Step 1: Clipped buildings of Hendrik-Ido-Ambacht
Step 2: Clipped roads of Hendrik-Ido-Ambacht
Step 3: Calculated street profile of Hendrik-Ido-Ambacht


  0%|          | 0/2347 [00:00<?, ?it/s]

Step 4: Calculated street orientation of Hendrik-Ido-Ambacht
Cleaning up...
Step 5: Stored street profile values in 'street_profiles[Hendrik-Ido-Ambacht]'.
----------------------------
Working on Rotterdam
----------------------------
Step 1: Clipped buildings of Rotterdam
Step 2: Clipped roads of Rotterdam
Step 3: Calculated street profile of Rotterdam


  0%|          | 0/35041 [00:00<?, ?it/s]

Step 4: Calculated street orientation of Rotterdam
Cleaning up...
Step 5: Stored street profile values in 'street_profiles[Rotterdam]'.
----------------------------
MISSION SUCCESS - END


##### Visualise your results

In [ ]:
# You can access the dataframes by giving the respective key. In this case, we're querying the street_profile dictionary using the key 'Rotterdam' to access the values.
# Then we can use any other regular function such as .explore()
street_profiles['Rotterdam'].explore(column='widths', cmap='Blues_r')

In [ ]:
# Same as above, just a different value
street_profiles['Rotterdam'].explore(column='orientation', cmap='viridis')

#### 2.3.3 Select trees in streets with e-w orientation.

In [21]:
# Select which orientation threshold you would like to use to filter the road network.
min_orientation = 80
max_orientation = 100

# Create an empty dictionary to save the tree selection in.
trees_canyons_dict = {}

# Loop through the study_areas_gdf to get results for each of the cities individually. The tqdm() wrapper is used to show the progress meter.
for index, row in tqdm(study_areas.iterrows(), total=study_areas.shape[0]):
        
        # Grab the area name to use later
        area_name = row['NAME_2']
        
        # To deal with the special character in the name
        if area_name == "\'s-Gravenhage":
            area_name = area_name[1:]
        
        # Iterate throught the items contained in the street_profiles dictionary. The city is the key, the edges are the streets. 
        for city, edges in street_profiles.items():
            
            # Make sure we're working with the right dataset by comparing city and area_name
            if city == area_name:
                print(f'Working on {city}')
                print('-----------------------')
                
                # Clip the tree dataset to the city extent
                trees_area = trees_other.clip(row.geometry)
                print('Step 1: Clipped tree dataset with city geometry')
                
                # Filter the road network to only include roads within the orientation values you defined above.
                e_w_streets = edges.loc[(edges.orientation >= min_orientation) & (edges.orientation <= max_orientation)]
                print(f'Step 2: Selected only roads between orientation of {min_orientation} and {max_orientation} degrees')
                
                # Create a copy of that dataframe
                e_w_streets_buffer = e_w_streets.copy()

                # Buffer edges with street width/2, without buffering the cap - we use this to filter out the trees in the roads.
                e_w_streets_buffer['geometry'] = e_w_streets_buffer.buffer((e_w_streets_buffer['widths']/2), cap_style=2)
                
                # Clipping with the buffer to select the trees that are in the roads.
                e_w_trees = trees_area.clip(e_w_streets_buffer)
                
                print(f'Results: In {city} {e_w_trees.shape[0]} trees are located in streets with east-west orientation. The filter eliminated {(trees_area.shape[0] - e_w_trees.shape[0])} trees from the dataset.')
                
                print('---------------------------')
                
                # Save the tree selection dataframe in the tree_dict using the current city as a key, similar to above.
                trees_canyons_dict[city] = e_w_trees
                
            else:
                # Makes sure that we continue until the city == area_name has found a match. 
                continue
print('MISSION SUCCESS - END')


  0%|          | 0/8 [00:00<?, ?it/s]

Working on Groningen
-----------------------
Step 1: Clipped tree dataset with city geometry
Step 2: Selected only roads between orientation of 80 and 100 degrees
Results: In Groningen 874 trees are located in streets with east-west orientation. The filter eliminated 8614 trees from the dataset.
---------------------------
Working on Amsterdam
-----------------------
Step 1: Clipped tree dataset with city geometry
Step 2: Selected only roads between orientation of 80 and 100 degrees
Results: In Amsterdam 6412 trees are located in streets with east-west orientation. The filter eliminated 21611 trees from the dataset.
---------------------------
Working on Heerhugowaard
-----------------------
Step 1: Clipped tree dataset with city geometry
Step 2: Selected only roads between orientation of 80 and 100 degrees
Results: In Heerhugowaard 352 trees are located in streets with east-west orientation. The filter eliminated 1300 trees from the dataset.
---------------------------
Working on Utre

##### Visualise your results

In [ ]:
m = street_profiles['Amsterdam'].explore(column="orientation", tooltip="widths", tiles="CartoDB positron", cmap="viridis"
)

trees_other.explore(
    m=m,
    color="red"
)

trees_canyons_dict['Amsterdam'].explore(
     m=m, # pass the map object
     color="green", # use red color on all points
)

folium.TileLayer('CartoDB positron', control=True).add_to(m)  # use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control
m.save("../results/output/trees_selection_e_w.html")

m  # show map

### 2.4 Floor Space Index (FSI) and Ground Space Index (GSI)

#### Calculate FSI

In [23]:
# Define a buffer area for FSI.
FSI_buffer = 50

# Call the function
floorspaceindexinbuffer(study_areas_gdf=study_areas, tree_dict=trees_canyons_dict, buildings_gdf=buildings_nl, buffersize=FSI_buffer)

  0%|          | 0/8 [00:00<?, ?it/s]

Working on Groningen
----------------------------
Step 1: Clipped buildings of Groningen
Step 2: Extracted trees of Groningen from the dictionary
Step 3: Created spatial index for Groningen buildings
Step 4: Calculated number of floors and cleaned the dataset
Step 5: For each tree, calculate FSI


  0%|          | 0/874 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
Working on Amsterdam
----------------------------
Step 1: Clipped buildings of Amsterdam
Step 2: Extracted trees of Amsterdam from the dictionary
Step 3: Created spatial index for Amsterdam buildings
Step 4: Calculated number of floors and cleaned the dataset
Step 5: For each tree, calculate FSI


  0%|          | 0/6412 [00:00<?, ?it/s]

...

Step 6: Add trees to the new dictionary
-------
Working on Hendrik-Ido-Ambacht
----------------------------
Step 1: Clipped buildings of Hendrik-Ido-Ambacht
Step 2: Extracted trees of Hendrik-Ido-Ambacht from the dictionary
Step 3: Created spatial index for Hendrik-Ido-Ambacht buildings
Step 4: Calculated number of floors and cleaned the dataset
Step 5: For each tree, calculate FSI


  0%|          | 0/22 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
Working on Rotterdam
----------------------------
Step 1: Clipped buildings of Rotterdam
Step 2: Extracted trees of Rotterdam from the dictionary
Step 3: Created spatial index for Rotterdam buildings
Step 4: Calculated number of floors and cleaned the dataset
Step 5: For each tree, calculate FSI


  0%|          | 0/2244 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
MISSION SUCCESS - END


In [24]:
trees_canyons_dict['Amsterdam'].head()

,rowid,uuid,bron,tree_name,tree_intid,genus,species,cv,bm_treeheight,bm_crown_diameter,geometry,description,FSI_50m
70958,653132,ab789f86-20a1-4063-aa77-6e317b749aa2,Amsterdam,Tilia ×europaea 'Pallida',1843,Tilia,europaea,Pallida,11.7,10.0,POINT (127552.757 478723.231),Residential,0.427712
11882,292656,4cc42d1c-eb8a-4177-a979-41b469eead6a,Amsterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,21.6,17.2,POINT (127783.315 478732.638),Residential,0.183039
34654,857787,e1282c82-2c16-43e0-9519-6662fb9e0d98,Amsterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,19.6,14.4,POINT (127791.515 478732.927),Residential,0.078593
56640,357881,5de8c1e8-ce6c-4663-a986-74ef6141a77e,Amsterdam,Tilia ×europaea 'Pallida',1843,Tilia,europaea,Pallida,12.5,12.9,POINT (126815.258 478212.550),Residential,0.247468
22072,547273,8f9d92a0-9424-4da0-86cf-e1fc7b844817,Amsterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,16.7,13.1,POINT (126287.323 478602.125),Residential,0.326963


In [ ]:
trees_canyons_dict['Amsterdam'].explore(column='FSI_50m', cmap='viridis', scheme='naturalbreaks')

#### Ground space index

Calculated for two tree dictionaries - separately:
- trees_canyons_dict 
- trees_in_parks_dict

In [26]:
## Define the buffer in m for GSI calculation
GSI_buffer_parks = 25


# Calls the groundspaceindexinbuffer function. NOTE: The results are saved in the tree_dict_FSI
groundspaceindexinbuffer_parks(study_areas_gdf=study_areas, tree_dict=trees_in_parks_dict, buildings_gdf=buildings_nl, buffersize=GSI_buffer_parks)

  0%|          | 0/8 [00:00<?, ?it/s]

Working on Groningen
----------------------------
Step 1: Clipped buildings of Groningen
Step 2: Extracted trees of Groningen from the dictionary
Step 3: Created spatial index for Groningen buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/349 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
Working on Amsterdam
----------------------------
Step 1: Clipped buildings of Amsterdam
Step 2: Extracted trees of Amsterdam from the dictionary
Step 3: Created spatial index for Amsterdam buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/795 [00:00<?, ?it/s]

...

Step 6: Add trees to the new dictionary
-------
Working on Hendrik-Ido-Ambacht
----------------------------
Step 1: Clipped buildings of Hendrik-Ido-Ambacht
Step 2: Extracted trees of Hendrik-Ido-Ambacht from the dictionary
Step 3: Created spatial index for Hendrik-Ido-Ambacht buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/8 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
Working on Rotterdam
----------------------------
Step 1: Clipped buildings of Rotterdam
Step 2: Extracted trees of Rotterdam from the dictionary
Step 3: Created spatial index for Rotterdam buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/350 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
MISSION SUCCESS - END


In [28]:
trees_in_parks_dict['Amsterdam'].head()

,rowid,uuid,bron,tree_name,tree_intid,genus,species,cv,bm_treeheight,bm_crown_diameter,geometry,description,tree_crown_area_25m,perc_tree_cover_25m,GSI_25m
90304,147061,26870aa1-1891-452c-99f6-5c05d3724760,Amsterdam,Ulmus ×hollandica 'Vegeta',1942,Ulmus,hollandica,Vegeta,26.2,19.4,POINT (114972.673 485394.808),Park and public garden,476.284888,0.242960,0.0
109242,898986,ec076801-e379-4008-aebf-c60eb9921e4c,Amsterdam,Ulmus ×hollandica 'Vegeta',1942,Ulmus,hollandica,Vegeta,24.4,15.2,POINT (114972.810 485403.708),Park and public garden,476.284888,0.242960,0.0
87636,42184,0b10443c-28ce-4f5f-bd5b-be0c8214ddc5,Amsterdam,Ulmus ×hollandica 'Vegeta',1942,Ulmus,hollandica,Vegeta,31.3,12.4,POINT (116154.413 485406.290),Park and public garden,1027.622872,0.524206,0.0
105761,758872,c73da659-c1e8-4333-8084-218486e4f7a9,Amsterdam,Ulmus ×hollandica 'Vegeta',1942,Ulmus,hollandica,Vegeta,3.0,9.5,POINT (116146.825 485412.136),Park and public garden,1179.573694,0.601718,0.0
110205,938059,f6439266-f5e8-4075-8df1-fc4f68f0aeab,Amsterdam,Ulmus ×hollandica 'Vegeta',1942,Ulmus,hollandica,Vegeta,40.4,14.1,POINT (116159.497 485412.481),Park and public garden,1010.476378,0.515459,0.0


In [27]:
## Define the buffer in m for GSI calculation
GSI_buffer = 25


# Calls the groundspaceindexinbuffer function. NOTE: The results are saved in the tree_dict_FSI
groundspaceindexinbuffer(study_areas_gdf=study_areas, tree_dict=trees_canyons_dict, buildings_gdf=buildings_nl, buffersize=GSI_buffer)

  0%|          | 0/8 [00:00<?, ?it/s]

Working on Groningen
----------------------------
Step 1: Clipped buildings of Groningen
Step 2: Extracted trees of Groningen from the dictionary
Step 3: Created spatial index for Groningen buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/874 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
Working on Amsterdam
----------------------------
Step 1: Clipped buildings of Amsterdam
Step 2: Extracted trees of Amsterdam from the dictionary
Step 3: Created spatial index for Amsterdam buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/6412 [00:00<?, ?it/s]

...

Step 6: Add trees to the new dictionary
-------
Working on Hendrik-Ido-Ambacht
----------------------------
Step 1: Clipped buildings of Hendrik-Ido-Ambacht
Step 2: Extracted trees of Hendrik-Ido-Ambacht from the dictionary
Step 3: Created spatial index for Hendrik-Ido-Ambacht buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/22 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
Working on Rotterdam
----------------------------
Step 1: Clipped buildings of Rotterdam
Step 2: Extracted trees of Rotterdam from the dictionary
Step 3: Created spatial index for Rotterdam buildings
Step 5: For each tree, calculate GSI


  0%|          | 0/2244 [00:00<?, ?it/s]

Step 6: Add trees to the new dictionary
-------
MISSION SUCCESS - END


In [29]:
trees_canyons_dict['Rotterdam'].head()

,rowid,uuid,bron,tree_name,tree_intid,genus,species,cv,bm_treeheight,bm_crown_diameter,geometry,description,FSI_50m,GSI_25m
18506,458858,786b4df9-8ce6-428b-8cc0-67c080b653d8,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,15.4,16.3,POINT (84842.250 429732.420),Residential,0.651120,0.025444
34378,850436,df3c07a9-dc53-4763-88a4-7af3c508ef0c,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,16.1,16.9,POINT (84856.260 429741.060),Residential,0.491170,0.059021
34690,858919,e17700eb-4ac0-40be-9db5-047087dc4d4c,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,13.2,15.1,POINT (85451.570 429962.560),Other agricultural usage,0.139921,0.000934
23200,575365,97071698-432d-4cd6-99c1-fa16fe5ea766,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,14.5,13.5,POINT (85465.500 429962.760),Other agricultural usage,0.116220,0.024233
12146,298867,4e65d3fd-b1e2-4da8-a589-29ecefba6ad4,Rotterdam,Platanus ×hispanica,1181,Platanus,hispanica,None,13.6,14.2,POINT (85479.410 429965.240),Other agricultural usage,0.067987,0.006934


In [ ]:
trees_canyons_dict['Rotterdam'].explore(column='GSI_25m')

## Selecting trees with the calculated attributes

Now the dataset is pre-filtered and contains FSI/GSI attributes. 

These can now be filtered further either in a pythonic way or in any external program.

The two dictionaries in question are:

trees_in_parks_dict: trees in parks

tree_dict_FSI_GSI: trees outside of parks

In [31]:
# Export trees in parks
for city, trees in tqdm(trees_in_parks_dict.items(), total=study_areas.shape[0]):
    try:
        trees.to_file(f'../results/output/tree_selection.gpkg', driver='GPKG', layer=f'{city}_parks')
    except:
        print(f'Error in data for {city}, e.g. no trees to save')

  0%|          | 0/8 [00:00<?, ?it/s]

In [32]:
# Export trees outside parks
for city, trees in tqdm(trees_canyons_dict.items(), total=study_areas.shape[0]):
    trees.to_file(f'../results/output/tree_selection.gpkg', driver='GPKG', layer=f'{city}_outside_parks')
            

  0%|          | 0/8 [00:00<?, ?it/s]